## Setup

In [1]:
import sys
import json
from pathlib import Path
import pandas as pd
import numpy as np
from collections import defaultdict

sys.path.append('../')

from src.predictors.team_predictor import rank_teams_for_track
from src.predictors.driver_predictor import DriverRanker
from src.utils.lineup_manager import get_lineups
from src.loaders.sessions import (
    extract_session_order_robust,
    test_session_as_predictor_fixed,
    calculate_order_mae
)
from src.predictors.blended_predictor import (
    get_fp_team_performance,
    blend_predictions
)

import warnings
warnings.filterwarnings('ignore')

import fastf1 as ff1
ff1.Cache.enable_cache('../data/raw/.fastf1_cache')

## Configuration

In [2]:
SEASON = 2025

print(f"Testing all prediction methods for {SEASON}")

Testing all prediction methods for 2025


## Load Data

In [3]:
# Track what loaded
loaded = []
errors = []

# Load track characteristics
try:
    track_path = Path(f'../data/processed/testing_files/track_characteristics/{SEASON}_track_characteristics.json')
    with open(track_path) as f:
        track_data = json.load(f)
    all_tracks = track_data.get('tracks', {})
    loaded.append(f"tracks ({len(all_tracks)})")
except FileNotFoundError:
    errors.append("track characteristics")
    all_tracks = {}

# Load car characteristics
try:
    car_path = Path(f'../data/processed/testing_files/car_characteristics/{SEASON}_car_characteristics.json')
    with open(car_path) as f:
        car_data = json.load(f)
    all_cars = car_data.get('teams', {})
    loaded.append(f"teams ({len(all_cars)})")
except FileNotFoundError:
    errors.append("car characteristics")
    all_cars = {}

# Load driver ranker
try:
    driver_ranker = DriverRanker(
        '../data/processed/testing_files/driver_characteristics/driver_characteristics.json'
    )
    loaded.append("driver ranker")
except FileNotFoundError:
    errors.append("driver characteristics")
    driver_ranker = None

# Load actual results
try:
    results_path = Path(f'../data/processed/testing_files/validation/{SEASON}_qualifying_results.json')
    with open(results_path) as f:
        actual_results = json.load(f)
    loaded.append(f"results ({actual_results.get('total_races', 0)} races)")
except FileNotFoundError:
    errors.append("qualifying results")
    actual_results = {}

# Print summary
if loaded:
    print(f"🟢 Loaded: {', '.join(loaded)}")
if errors:
    print(f"🔴  Missing: {', '.join(errors)}")

Loaded characteristics for 27 drivers
🟢 Loaded: tracks (24), teams (10), driver ranker, results (24 races)


## Test 1: Session Orders as Predictors

**THE CRITICAL TEST - NOW WITH LAP TIMES!**

In [4]:
print("TEST 1: Session Orders as Predictors (USING LAP TIMES)")
print("="*70)
print("\nExtracting session orders from all races...")
print("(This may take 5-10 minutes - loading lap data)\n")

# Store results
session_results = defaultdict(list)
extraction_stats = defaultdict(int)

for race_name, race_data in actual_results['races'].items():
    if race_name not in all_tracks:
        continue
    
    weekend_type = race_data['weekend_type']
    lineups = get_lineups(SEASON, race_name)
    
    # Determine which sessions to test
    if weekend_type == 'sprint':
        sessions_to_test = ['FP1', 'Sprint Qualifying']
    else:
        sessions_to_test = ['FP1', 'FP2', 'FP3']
    
    # Test each session
    for session in sessions_to_test:
        result = test_session_as_predictor_fixed(
            SEASON,
            race_name,
            session,
            'Q',
            driver_ranker,
            lineups,
            race_data['positions']
        )
        
        if result['status'] == 'success':
            extraction_stats[session] += 1
            
            if 'driver_mae' in result:
                session_results[session].append({
                    'race': race_name,
                    'weekend_type': weekend_type,
                    'team_mae': result['team_mae'],
                    'driver_mae': result['driver_mae']
                })
                print(f"🟢 {race_name:<30} {session:<20} Driver MAE: {result['driver_mae']:.2f}")
            else:
                print(f"⚠ {race_name:<30} {session:<20} Team OK, Driver failed")
        else:
            print(f"🔴 {race_name:<30} {session:<20} {result['reason']}")

# Show extraction stats
print(f"\n{'='*70}")
print("EXTRACTION STATS")
print("-"*70)
for session in ['FP1', 'FP2', 'FP3', 'Sprint Qualifying']:
    if session in extraction_stats:
        print(f"{session:<20} {extraction_stats[session]} races extracted")
    else:
        print(f"{session:<20} 0 races extracted")

# Aggregate results
print(f"\n{'='*70}")
print("RESULTS: Session Orders")
print("-"*70)
print(f"{'Session':<20} {'Races':<8} {'Team MAE':<12} {'Driver MAE':<12}")
print("-"*70)

for session in ['FP1', 'FP2', 'FP3', 'Sprint Qualifying']:
    if session in session_results and session_results[session]:
        results = session_results[session]
        avg_team_mae = np.mean([r['team_mae'] for r in results])
        avg_driver_mae = np.mean([r['driver_mae'] for r in results])
        
        print(f"{session:<20} {len(results):<8} {avg_team_mae:<12.2f} {avg_driver_mae:<12.2f}")
    else:
        print(f"{session:<20} {'0':<8} {'N/A':<12} {'N/A':<12}")

TEST 1: Session Orders as Predictors (USING LAP TIMES)

Extracting session orders from all races...
(This may take 5-10 minutes - loading lap data)

🟢 Australian Grand Prix          FP1                  Driver MAE: 3.53
🟢 Australian Grand Prix          FP2                  Driver MAE: 3.18
🟢 Australian Grand Prix          FP3                  Driver MAE: 4.00
🟢 Chinese Grand Prix             FP1                  Driver MAE: 3.05
🟢 Chinese Grand Prix             Sprint Qualifying    Driver MAE: 2.82
🟢 Japanese Grand Prix            FP1                  Driver MAE: 3.94
🟢 Japanese Grand Prix            FP2                  Driver MAE: 4.26
🟢 Japanese Grand Prix            FP3                  Driver MAE: 3.31
🟢 Bahrain Grand Prix             FP1                  Driver MAE: 5.70
🟢 Bahrain Grand Prix             FP2                  Driver MAE: 3.74
🟢 Bahrain Grand Prix             FP3                  Driver MAE: 3.84
🟢 Saudi Arabian Grand Prix       FP1                  Driver MAE: 3.39

## Test 2: Model-Only Predictions

In [5]:
print("\nTEST 2: Model-Only Predictions (Baseline)")
print("="*70)

model_results = []

for race_name, race_data in actual_results['races'].items():
    if race_name not in all_tracks:
        continue
    
    track = all_tracks[race_name]
    weekend_type = race_data['weekend_type']
    lineups = get_lineups(SEASON, race_name)
    
    if weekend_type == 'sprint':
        session = 'post_sprint_quali'
    else:
        session = 'post_fp3'
    
    team_rankings = rank_teams_for_track(all_cars, track, session, weekend_type)
    if not team_rankings:
        continue
    
    model_ranks = {team: rank for rank, (team, _, _, _) in enumerate(team_rankings, 1)}
    
    actual_team_order = extract_session_order_robust(SEASON, race_name, 'Q')
    if not actual_team_order:
        continue
    
    team_mae = calculate_order_mae(model_ranks, actual_team_order)
    
    driver_preds = driver_ranker.predict_positions(
        team_predictions=model_ranks,
        team_lineups=lineups,
        session_type='qualifying'
    )
    
    errors = []
    for pred in driver_preds['predictions']:
        actual_pos = next(
            (p['position'] for p in race_data['positions'] if p['driver'] == pred.driver),
            None
        )
        if actual_pos:
            errors.append(abs(pred.position - actual_pos))
    
    if errors:
        model_results.append({
            'race': race_name,
            'weekend_type': weekend_type,
            'team_mae': team_mae,
            'driver_mae': np.mean(errors)
        })

if model_results:
    avg_team_mae = np.mean([r['team_mae'] for r in model_results])
    avg_driver_mae = np.mean([r['driver_mae'] for r in model_results])
    
    print(f"\nModel-Only (Blind Prediction):")
    print(f"  Races: {len(model_results)}")
    print(f"  Team MAE: {avg_team_mae:.2f} positions")
    print(f"  Driver MAE: {avg_driver_mae:.2f} positions")


TEST 2: Model-Only Predictions (Baseline)

Model-Only (Blind Prediction):
  Races: 24
  Team MAE: 1.85 positions
  Driver MAE: 4.30 positions


## Test 3: Blended Predictions

In [6]:
print("\nTEST 3: Blended Predictions (Model + FP Data)")
print("="*70)

blend_weights = [0.3, 0.5, 0.7, 0.9]
blend_results = {weight: [] for weight in blend_weights}

for race_name, race_data in actual_results['races'].items():
    if race_name not in all_tracks:
        continue
    
    track = all_tracks[race_name]
    weekend_type = race_data['weekend_type']
    lineups = get_lineups(SEASON, race_name)
    
    if weekend_type == 'sprint':
        session = 'post_sprint_quali'
        fp_session = 'Sprint Qualifying'
    else:
        session = 'post_fp3'
        fp_session = 'FP3'
    
    team_rankings = rank_teams_for_track(all_cars, track, session, weekend_type)
    if not team_rankings:
        continue
    
    model_ranks = {team: rank for rank, (team, _, _, _) in enumerate(team_rankings, 1)}
    
    fp_performance = get_fp_team_performance(SEASON, race_name, fp_session)
    if not fp_performance:
        continue
    
    actual_team_order = extract_session_order_robust(SEASON, race_name, 'Q')
    if not actual_team_order:
        continue
    
    for weight in blend_weights:
        blended_ranks = blend_predictions(model_ranks, fp_performance, weight_fp=weight)
        
        team_mae = calculate_order_mae(blended_ranks, actual_team_order)
        
        driver_preds = driver_ranker.predict_positions(
            team_predictions=blended_ranks,
            team_lineups=lineups,
            session_type='qualifying'
        )
        
        errors = []
        for pred in driver_preds['predictions']:
            actual_pos = next(
                (p['position'] for p in race_data['positions'] if p['driver'] == pred.driver),
                None
            )
            if actual_pos:
                errors.append(abs(pred.position - actual_pos))
        
        if errors:
            blend_results[weight].append({
                'race': race_name,
                'weekend_type': weekend_type,
                'team_mae': team_mae,
                'driver_mae': np.mean(errors)
            })

# Aggregate (FIXED DISPLAY)
print(f"\n{'Weight (FP/Model)':<20} {'Races':<8} {'Team MAE':<12} {'Driver MAE':<12}")
print("-"*70)

for weight in blend_weights:
    if blend_results[weight]:
        results = blend_results[weight]
        avg_team_mae = np.mean([r['team_mae'] for r in results])
        avg_driver_mae = np.mean([r['driver_mae'] for r in results])
        
        # FIXED DISPLAY
        fp_pct = int(weight * 100)
        model_pct = 100 - fp_pct
        label = f"{fp_pct}/{model_pct}"
        
        print(f"{label:<20} {len(results):<8} {avg_team_mae:<12.2f} {avg_driver_mae:<12.2f}")


TEST 3: Blended Predictions (Model + FP Data)

Weight (FP/Model)    Races    Team MAE     Driver MAE  
----------------------------------------------------------------------
30/70                24       1.65         3.96        
50/50                24       1.48         3.78        
70/30                24       1.44         3.65        
90/10                24       1.43         3.60        


## Overall Comparison

In [7]:
print("\n" + "="*70)
print("OVERALL COMPARISON - DRIVER MAE (Lower = Better)")
print("="*70)

comparison = []

# Session orders
for session in ['FP1', 'FP2', 'FP3', 'Sprint Qualifying']:
    if session in session_results and session_results[session]:
        results = session_results[session]
        avg_mae = np.mean([r['driver_mae'] for r in results])
        comparison.append({
            'method': f'{session} order',
            'driver_mae': avg_mae,
            'races': len(results),
            'type': 'session'
        })

# Model-only
if model_results:
    avg_mae = np.mean([r['driver_mae'] for r in model_results])
    comparison.append({
        'method': 'Model-only',
        'driver_mae': avg_mae,
        'races': len(model_results),
        'type': 'model'
    })

# Blended
for weight in blend_weights:
    if blend_results[weight]:
        results = blend_results[weight]
        avg_mae = np.mean([r['driver_mae'] for r in results])
        
        fp_pct = int(weight * 100)
        model_pct = 100 - fp_pct
        label = f"Blend {fp_pct}/{model_pct}"
        
        comparison.append({
            'method': label,
            'driver_mae': avg_mae,
            'races': len(results),
            'type': 'blend'
        })

# Sort by MAE
comparison.sort(key=lambda x: x['driver_mae'])

print(f"\n{'Rank':<6} {'Method':<25} {'Driver MAE':<12} {'Races':<8}")
print("-"*70)

for rank, result in enumerate(comparison, 1):
    status = "🥇" if rank == 1 else "🥈" if rank == 2 else "🥉" if rank == 3 else "  "
    print(f"{status} {rank:<3} {result['method']:<25} {result['driver_mae']:<12.2f} {result['races']:<8}")

# Winner
if comparison:
    winner = comparison[0]
    baseline = next((c for c in comparison if c['method'] == 'Model-only'), None)
    
    if baseline:
        improvement = baseline['driver_mae'] - winner['driver_mae']
        pct_improvement = (improvement / baseline['driver_mae']) * 100
        
        print(f"\n" + "="*70)
        print(f"🏆 WINNER: {winner['method']}")
        print("="*70)
        print(f"  Driver MAE: {winner['driver_mae']:.2f} positions")
        print(f"  Improvement over model: {improvement:+.2f} positions ({pct_improvement:+.1f}%)")
        print(f"  Tested on: {winner['races']} races")
        print(f"  Type: {winner['type']}")


OVERALL COMPARISON - DRIVER MAE (Lower = Better)

Rank   Method                    Driver MAE   Races   
----------------------------------------------------------------------
🥇 1   Sprint Qualifying order   3.22         6       
🥈 2   Blend 90/10               3.60         24      
🥉 3   Blend 70/30               3.65         24      
   4   Blend 50/50               3.78         24      
   5   FP3 order                 3.90         18      
   6   Blend 30/70               3.96         24      
   7   FP2 order                 4.19         18      
   8   Model-only                4.30         24      
   9   FP1 order                 4.71         24      

🏆 WINNER: Sprint Qualifying order
  Driver MAE: 3.22 positions
  Improvement over model: +1.08 positions (+25.1%)
  Tested on: 6 races
  Type: session


## KEY COMPARISON: Session Orders vs Blending

In [8]:
print("\n" + "="*70)
print("KEY COMPARISON: Session Orders vs Blending")
print("="*70)

session_approaches = [c for c in comparison if c['type'] == 'session']
blend_approaches = [c for c in comparison if c['type'] == 'blend']

if session_approaches:
    best_session = min(session_approaches, key=lambda x: x['driver_mae'])
    print(f"\nBEST SESSION ORDER:")
    print(f"   {best_session['method']}: {best_session['driver_mae']:.2f} MAE ({best_session['races']} races)")

if blend_approaches:
    best_blend = min(blend_approaches, key=lambda x: x['driver_mae'])
    print(f"\nBEST BLEND:")
    print(f"   {best_blend['method']}: {best_blend['driver_mae']:.2f} MAE ({best_blend['races']} races)")

if session_approaches and blend_approaches:
    diff = best_blend['driver_mae'] - best_session['driver_mae']
    
    print(f"\n" + "-"*70)
    print(f"VERDICT:")
    print("-"*70)
    
    if diff > 0.1:
        print(f"🟢 SESSION ORDER WINS by {diff:.2f} positions!")
        print(f"\n   Recommendation: Use {best_session['method']}")
        print(f"   Expected MAE: {best_session['driver_mae']:.2f}")
        print(f"\n   Why: Simpler, more accurate, no model needed!")
    elif diff < -0.1:
        print(f"🟢 BLENDING WINS by {-diff:.2f} positions!")
        print(f"\n   Recommendation: Use {best_blend['method']}")
        print(f"   Expected MAE: {best_blend['driver_mae']:.2f}")
        print(f"\n   Why: Worth the extra complexity!")
    else:
        print(f"🟢🔴 TIE! Difference: {abs(diff):.2f} positions")
        print(f"\n   Both approaches work equally well!")
        print(f"   Recommendation: Use {best_session['method']} (simpler)")
elif session_approaches:
    print(f"\n🟢 Session orders extracted successfully!")
    print(f"   Best: {best_session['method']} - {best_session['driver_mae']:.2f} MAE")
elif blend_approaches:
    print(f"\n🔴 Session orders failed to extract!")
    print(f"   Best blend: {best_blend['method']} - {best_blend['driver_mae']:.2f} MAE")


KEY COMPARISON: Session Orders vs Blending

BEST SESSION ORDER:
   Sprint Qualifying order: 3.22 MAE (6 races)

BEST BLEND:
   Blend 90/10: 3.60 MAE (24 races)

----------------------------------------------------------------------
VERDICT:
----------------------------------------------------------------------
🟢 SESSION ORDER WINS by 0.38 positions!

   Recommendation: Use Sprint Qualifying order
   Expected MAE: 3.22

   Why: Simpler, more accurate, no model needed!


## Save Results

In [9]:
output = {
    'season': SEASON,
    'test_date': pd.Timestamp.now().isoformat(),
    'session_orders': {
        session: {
            'races': len(results),
            'avg_driver_mae': float(np.mean([r['driver_mae'] for r in results])),
            'avg_team_mae': float(np.mean([r['team_mae'] for r in results]))
        }
        for session, results in session_results.items()
    },
    'model_only': {
        'races': len(model_results),
        'avg_driver_mae': float(np.mean([r['driver_mae'] for r in model_results])),
        'avg_team_mae': float(np.mean([r['team_mae'] for r in model_results]))
    } if model_results else None,
    'blended': {
        f"{int(w*100)}_{int((1-w)*100)}": {
            'races': len(results),
            'avg_driver_mae': float(np.mean([r['driver_mae'] for r in results])),
            'avg_team_mae': float(np.mean([r['team_mae'] for r in results]))
        }
        for w, results in blend_results.items() if results
    },
    'winner': {
        'method': winner['method'],
        'driver_mae': float(winner['driver_mae']),
        'type': winner['type']
    } if comparison else None
}

output_path = Path(f'../data/processed/testing_files/validation/comprehensive_test_{SEASON}.json')
output_path.parent.mkdir(parents=True, exist_ok=True)

with open(output_path, 'w') as f:
    json.dump(output, f, indent=2)

print(f"\n🟢 Saved results to {output_path}")


🟢 Saved results to ../data/processed/testing_files/validation/comprehensive_test_2025.json
